<a href="https://colab.research.google.com/github/prasanna-venkatesh-m/RNN-Basics/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [10]:
data = pd.read_excel('/content/Feb data.xlsx')

In [11]:
data.head()

,Date,Accounts
0,2024-02-01,16186
1,2024-02-02,16235
2,2024-02-05,16369
3,2024-02-06,16466
4,2024-02-07,16469


In [13]:
values = data['Accounts'].values.reshape(-1,1)

In [16]:
values.shape

(23, 1)

In [54]:
window = 7

def create_sequences(data, window):
    X, y = [], []
    for i in range(len(data) - window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled, window)

X = X.reshape((X.shape[0], X.shape[1], 1))

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (16, 7, 1)
y shape: (16, 1)


In [18]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaled = scaler.fit_transform(values)

In [69]:
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(10, input_shape=(window,1)),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [55]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [56]:
model.fit(x_train, y_train, epochs=200)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0198
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0196
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0195
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0193
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0191
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0189
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0187
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0185
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0184
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0182
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0181
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0179
Epoch 13/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0178
Epoch 14/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0176
Epoch 15/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0175
Epoch 16/200
1/1 ━━

In [57]:
y_pred_scaled = model.predict(x_test)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_actual = scaler.inverse_transform(y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step


In [75]:
print(mean_squared_error(y_pred, y_test_actual))
print(mean_absolute_error(y_pred, y_test_actual))

3393.388571739197
40.60107421875


In [66]:
print(y_test_actual)
print(y_pred)

[[16979.]
 [16903.]
 [16502.]
 [16620.]]
[[16978.848]
 [16859.068]
 [16609.346]
 [16630.975]]


In [39]:
last_seq = X[-1].reshape(1, window, 1)

In [40]:
last_seq.shape

(1, 7, 1)

In [41]:
pred_scaled = model.predict(last_seq)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step


In [43]:
pred = scaler.inverse_transform(pred_scaled)

In [44]:
pred

array([[16908.023]], dtype=float32)

In [45]:
new_seq = np.append(last_seq[:,1:,:], [[pred_scaled[0]]], axis=1)

In [47]:
last_seq

array([[[0.71752837],
        [0.71500631],
        [0.97982346],
        [0.44010088],
        [0.82345523],
        [0.80832282],
        [1.        ]]])

In [46]:
new_seq

array([[[0.71500631],
        [0.97982346],
        [0.44010088],
        [0.82345523],
        [0.80832282],
        [1.        ],
        [0.82098901]]])

In [49]:
pred_scaled_2 = model.predict(new_seq)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [50]:
next_pred = scaler.inverse_transform(pred_scaled_2)

In [51]:
next_pred

array([[16947.621]], dtype=float32)